In [30]:
import numpy as np # linear algebra
import pandas as pd

poetry = pd.read_csv('all.csv')
num_poems = 573

train=poetry.sample(frac=0.8726,random_state=200)
test=poetry.drop(train.index)

poem = train.iloc[:,1]
poems = []
for item in poem:
    poems.append(item)
    
y = train.iloc[:, 4]
one_hot = pd.get_dummies(y)
labels = one_hot.values

poem1 = test.iloc[:,1]
poems1 = []
for item in poem1:
    poems1.append(item)
    
y1 = test.iloc[:, 4]
one_hot = pd.get_dummies(y1)
labels1 = one_hot.values

poem_train = poems
poem_test = poems1

train_y = labels
test_y = labels1

#Model
import tensorflow as tf
import numpy as np

# We need to import several things from Keras.
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU,LSTM, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

#Tokenization
num_words = 2000 # setting the max number of words = vocabulary
tokenizer = Tokenizer(num_words = num_words) # creating the tokenizer
data_text = poem_train + poem_test # creating our tokenizer with the entire dataset

tokenizer.fit_on_texts(data_text) # fitting the tokenizer to data
train_x_tokens = tokenizer.texts_to_sequences(poem_train)
test_x_tokens = tokenizer.texts_to_sequences(poem_test)

In [31]:
num_tokens = [len(tokens) for tokens in train_x_tokens + test_x_tokens]
num_tokens = np.array(num_tokens)

# The max number of tokens is set to the average plus 2 standard deviations
max_tokens = np.mean(num_tokens)+ 2 * np.std(num_tokens)

#Converting the value to int
max_tokens = int(max_tokens)
print(max_tokens)

pad = 'pre' # add 0's to the first places of the array
train_x_pad = pad_sequences(train_x_tokens, maxlen = max_tokens, padding= pad, truncating = pad)
test_x_pad = pad_sequences(test_x_tokens, maxlen=max_tokens , padding = pad, truncating = pad)

# Tokenizer Inverse Map
#  a function that will translate the integer-tokens back to words

idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

# Helper function for converting a list of tokends back to a string of words

def tokens_to_string(tokens):
    # Map from tokens back to the words:
    words = [inverse_map[token] for token in tokens if token != 0]
    
    #Concatenate all the words
    text = " ".join(words)
    return text

700


In [47]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim = num_words, output_dim = embedding_size
                    , input_length = max_tokens, name = 'layer_embedding'))
model.add(LSTM(25, return_sequences = True))
model.add(LSTM(15))
model.add(Dense(3,activation='softmax'))
optimizer = Adam(lr=1e-2)
model.compile(loss ='categorical_crossentropy', optimizer =  optimizer, metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 700, 8)            16000     
_________________________________________________________________
lstm_20 (LSTM)               (None, 700, 25)           3400      
_________________________________________________________________
lstm_21 (LSTM)               (None, 15)                2460      
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 48        
Total params: 21,908
Trainable params: 21,908
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(train_x_pad, train_y, validation_split = 0.05, epochs = 8, batch_size = 64)

Train on 475 samples, validate on 25 samples
Epoch 1/8
475/475 [==============================] - 13s 28ms/step - loss: 0.9537 - acc: 0.5558 - val_loss: 0.7655 - val_acc: 0.6800
Epoch 2/8
475/475 [==============================] - 6s 14ms/step - loss: 0.9410 - acc: 0.5537 - val_loss: 0.7919 - val_acc: 0.6800
Epoch 3/8
475/475 [==============================] - 6s 13ms/step - loss: 0.9287 - acc: 0.5537 - val_loss: 0.8088 - val_acc: 0.6800
Epoch 4/8
475/475 [==============================] - 6s 13ms/step - loss: 0.9136 - acc: 0.5558 - val_loss: 0.7952 - val_acc: 0.7200
Epoch 5/8
475/475 [==============================] - 6s 12ms/step - loss: 0.8210 - acc: 0.6695 - val_loss: 0.7678 - val_acc: 0.7200
Epoch 6/8
475/475 [==============================] - 6s 13ms/step - loss: 0.7248 - acc: 0.7179 - val_loss: 0.8134 - val_acc: 0.7200
Epoch 7/8
475/475 [==============================] - 6s 12ms/step - loss: 0.6778 - acc: 0.7347 - val_loss: 0.8752 - val_acc: 0.6800
Epoch 8/8
475/475 [===========

In [49]:
result = model.evaluate(test_x_pad, test_y)
print("Accuracy : {0:.2%}".format(result[1]))

73/73 [==============================] - 0s 4ms/step
Accuracy : 68.49%
